In [1]:
import os

In [2]:
%pwd

'e:\\PROJECTS\\Loan_Approval_Prediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\PROJECTS\\Loan_Approval_Prediction'

In [5]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/Atanu-Manna2003/Loan_Approval_Prediction.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "Atanu-Manna2003"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "d2f7245736c5703adf96b18a0b4d4a37d4d2cbd3"  # <- Your Access Token


In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    scaler_path:Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [7]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.LogisticRegression
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            scaler_path=config.scaler_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/Atanu-Manna2003/Loan_Approval_Prediction.mlflow",
           
        )

        return model_evaluation_config


In [9]:
import pandas as pd
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
from pathlib import Path
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from mlProject.utils.common import save_json


In [10]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        acc = accuracy_score(actual, pred)
        precision = precision_score(actual, pred, average="binary")
        recall = recall_score(actual, pred, average="binary")
        f1 = f1_score(actual, pred, average="binary")
        try:
            roc_auc = roc_auc_score(actual, pred)
        except:
            roc_auc = None
        return acc, precision, recall, f1, roc_auc

    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        # ✅ Load scaler
        scaler = joblib.load(self.config.scaler_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[self.config.target_column]

        # ✅ Scale test data
        test_x_scaled = scaler.transform(test_x)

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_labels = model.predict(test_x_scaled)

            (acc, precision, recall, f1, roc_auc) = self.eval_metrics(test_y, predicted_labels)

            scores = {
                "accuracy": acc,
                "precision": precision,
                "recall": recall,
                "f1_score": f1,
                "roc_auc": roc_auc,
            }
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("accuracy", acc)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall", recall)
            mlflow.log_metric("f1_score", f1)
            if roc_auc is not None:
                mlflow.log_metric("roc_auc", roc_auc)

            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="LoanApprovalModel")
            else:
                mlflow.sklearn.log_model(model, "model")


In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-08-31 21:05:55,860: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-31 21:05:55,868: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-31 21:05:55,875: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-08-31 21:05:55,875: INFO: common: created directory at: artifacts]
[2025-08-31 21:05:55,875: INFO: common: created directory at: artifacts/model_evaluation]
[2025-08-31 21:05:57,477: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


e:\PROJECTS\Loan_Approval_Prediction\myenv\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'LoanApprovalModel' already exists. Creating a new version of this model...
2025/08/31 21:06:16 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: LoanApprovalModel, version 2
Created version '2' of model 'LoanApprovalModel'.
